$L = K_0 \times \dotsb K_m$, where each $K_i/k$ is a cyclic extension of number field with degree $p^{\epsilon_i}$. These fields satisfies that
* $K_i \not\subset K_j$ for all $i, j$.
* $\cap_{i \in \mathcal{I}'} K_i = k$.
* $e_i \geq e_{i+1}$.

Input: the cyclic extensions $K_1, \dotsc K_m$ and $k$.

Ouput: the groups $Ш(k, \hat{T}_{L/k})$ and $Ш_{\omega}(k, \hat{T}_{L/k})$.

# Inputs

In [1]:
# use example 7.4
m = 2
I = [i for i in range(1, m+1)]
I_prime = [i for i in range(0, m+1)]

In [2]:
k = NumberField(x^2 + 1, 'i')

In [3]:
K = [0] * (m+1)

In [4]:
K[0] = k.extension(x^4 - 17, 'a0')

In [5]:
K[1] = k.extension(x^4 - 221, 'a1')

In [6]:
K[2] = k.extension(x^4 - 13, 'a2')

In [7]:
p = 2

Find $\epsilon_i$, where $p^{\epsilon_i} = [K_i:k]$.

In [8]:
from sage.functions.log import logb
epsilon = [logb(K[i].absolute_degree()/k.absolute_degree(), p) for i in range(0, m+1)]

In [9]:
epsilon

[log(4)/log(2), log(4)/log(2), log(4)/log(2)]

Find $e_i$ and $e_{i, j}$, where $p^{e_i} = [K_0 K_i : K_i]$ and $p^{e_{i, j}} = [K_i \cap K_j: k]$.

In [10]:
e_ij = [
    [epsilon[i] - int(logb(K[i].composite_fields(K[j])[0].absolute_degree()/K[j].absolute_degree(), p))
     for j in range(0, m+1)]
    for i in range(0, m+1)
]

In [11]:
e_ij

[[log(4)/log(2), log(4)/log(2) - 2, log(4)/log(2) - 2],
 [log(4)/log(2) - 2, log(4)/log(2), log(4)/log(2) - 2],
 [log(4)/log(2) - 2, log(4)/log(2) - 2, log(4)/log(2)]]

In [12]:
e_i = [epsilon[0] - e_ij[0][i] for i in range(0, m+1)]

In [13]:
e_i

[0, 2, 2]

$R = \{i \in \mathcal{I}'\mid U_i \neq \varnothing\}$, where $U_r = \{i \in \mathcal{I} \mid e_{0, i} = r\}$.

In [14]:
def U(r):
    result = []
    for i in I:
        if e_ij[0][i] == r:
            result.append(i)
    return result

R = []
for i in I_prime:
    if len(U(i)) > 0:
        R.append(i)

In [15]:
R

[0]

In [16]:
U(0)

[1, 2]

In [17]:
U(1)

[]

In [18]:
U(2)

[]

$L(U_r) = \min\{\log_p[K_i \cap K_j: k] \mid \forall i, j \in U_r\}$

In [19]:
def L(U):
    r = []
    for i in U:
        for j in U:
            r.append(e_ij[i][j])
    if len(r) > 0:
        return min(r)
    else:
        return 'The set is empty!'

In [20]:
L(U(0))

log(4)/log(2) - 2

On a subset $C \subset \mathcal{I}'$, we can define an equivalence relation $\sim_{\ell}$ by
$$i \sim_{\ell} j \Leftrightarrow e_{ij} \geq l.$$
The number $n_{\ell}(C)$ is the number of equivalence classes with respect to $C/\sim_{\ell}$.

In [21]:
def equiv(i, j, l):
    return e_ij[i][j] >= l or i == j

In [22]:
def equiv_classes(C, l):
    # return a list of equivalence classes with respect to C/l
    d = {}
    for i in C:
        connected_vertices = []
        for j in C:
            if i != j and equiv(i, j, l):
                connected_vertices.append(j)
        d[i] = connected_vertices
    G = Graph(d)
    return G.connected_components()

In [23]:
equiv_classes(I_prime, 0)

[[0, 1, 2]]

In [24]:
equiv_classes(I_prime, 1)

[[0], [1], [2]]

In [25]:
def n(l, C):
    d = {}
    for i in C:
        connected_vertices = []
        for j in C:
            if i != j and equiv(i, j, l):
                connected_vertices.append(j)
        d[i] = connected_vertices
    G = Graph(d)
    return G.connected_components_number()

For a cyclic extension $K_i$ of degree $p^{\epsilon_i}$, we denote by $K_i(f)$ the unique subfield of $K_i$ of degree $p^f$ (p4).

In [26]:
def is_subfield(k, K):
    # check that if k is a subfield of K
    # assume k is Galois
    R.<x> = PolynomialRing(K)
    f = R(k.polynomial())
    l = list(f.factor())
    flag = True
    for factor in l:
        if factor[0].degree() != 1:
            flag = False
            break
    return flag

In [27]:
def K_cyclic_subfield(i, d):
    # return the field K_i(d)
    L_list = K[i].subfields(p^d * k.degree())
    subfield_list = [subfield[0] for subfield in L_list]
    for subfield in subfield_list:
        if is_subfield(k, subfield):
            return subfield

In [28]:
K_cyclic_subfield(0, 1)

Number Field in a0_0 with defining polynomial x^4 + 8*x^3 - 104*x^2 - 480*x + 4688

For a nonempty subset $C \subseteq \mathcal{I}$ and an integer $d \geq 0$, we define the field $M_C(d)$ to be the composite field $\langle K_i(d) \rangle_{i \in C}$.

In [29]:
def M_composite(C, d):
    result = k
    for i in C:
        result = result.composite_fields(K_cyclic_subfield(i, d))[0]
    result = k.composite_fields(result)[0]
    return result

Testing

In [30]:
I

[1, 2]

In [31]:
K_cyclic_subfield(1, 1)

Number Field in a1_0 with defining polynomial x^4 + 8*x^3 - 1736*x^2 - 7008*x + 781520

In [32]:
K_cyclic_subfield(2, 1)

Number Field in a2_0 with defining polynomial x^4 + 8*x^3 - 72*x^2 - 352*x + 2768

## Definition 4.1.
The *algebraic patching degree* $\Delta^{\omega}_r$ of $U_r$ is the maximum nonnegative integer $d$ satisfying the following:
1. If $U_{>r}$ is nonempty, then $M_{U_{>r}}(d) \subset \cap_{i \in U_r} K_0(d) K_i(d)$.
2. If $U_{<r}$ is nonempty, then $M_{U_r}(d) \subset \cap_{i \in U_{<r}} K_0(d) K_i(d)$.

If $U_r = \mathcal{I}$, then we set $\Delta^{\omega}_r = \epsilon_0$.

Here $U_{> r} = \{i \in \mathcal{I} \mid e_{0, i} > r\}$ and $U_{< r} = \{i \in \mathcal{I} \mid e_{0, i} < r\}$. (p.13)

In [33]:
def U_greater(r):
    result = []
    for i in I:
        if e_ij[0][i] > r:
            result.append(i)
    return result

def U_smaller(r):
    result = []
    for i in I:
        if e_ij[0][i] < r:
            result.append(i)
    return result

In [34]:
def Delta_omega(r):
    # the algebraic patching degree of U_r
    U_r = U(r)
    if U_r == I:
        return epsilon[0]
    
    U_greater_r = U_greater(r)
    U_smaller_r = U_smaller(r)
    
    flag_1 = bool(len(U_greater(r)) > 0)
    flag_2 = bool(len(U_smaller(r)) > 0)
    
    for d in range(r, epsilon[0]+1):
        d_satisfy = True
        
        if flag_1:
            for i in U_r:
                temp_field = K_cyclic_subfield(0, d).composite_fields(K_cyclic_subfield(i, d))[0]
                if not is_subfield(M_composite(U_greater_r, d), temp_field):
                    d_satisfy = False
                    break
        if flag_2:
            for i in U_smaller_r:
                temp_field = K_cyclic_subfield(0, d).composite_fields(K_cyclic_subfield(i, d))[0]
                if not is_subfield(M_composite(U_r, d), temp_field):
                    d_satisfy = False
                    break
        if not d_satisfy:
            return d-1
    return epsilon[0]

In [35]:
Delta_omega(0)

log(4)/log(2)

A finite Galois extension $F$ of $k$ is said to be *locally cyclic* at $v$ if $F \otimes_k k_v$ is a product of cyclic extensions of $k_v$. $F$ is said to be *locally cyclic* if it is locally cyclic at all $v \in \Omega_k$. (p2)

In [36]:
temp_k.<s> = NumberField(x^2 + 1)

In [37]:
type(temp_k)

<class 'sage.rings.number_field.number_field.NumberField_quadratic_with_category'>

In [38]:
temp_K.<t> = temp_k.extension(x^3 + 7)
type(temp_K)

<class 'sage.rings.number_field.number_field_rel.NumberField_relative_with_category'>

In [39]:
t

t

In [40]:
type(temp_K.absolute_field('w'))

<class 'sage.rings.number_field.number_field.NumberField_absolute_with_category'>

In [41]:
temp_K.absolute_field('w')

Number Field in w with defining polynomial x^6 + 3*x^4 + 14*x^3 + 3*x^2 - 42*x + 50

In [42]:
def is_locally_cyclic(F):
    if is_subfield(F, k):
        return True
    
    F_abs.<w> = F.absolute_field()
    k_gen = (x^2 + 1).roots(ring = F_abs, multiplicities = False)[0]
    G_abs = F_abs.galois_group()
    rel_disc = k.ideal(F.relative_discriminant())
    temp_list = list(F.ideal(rel_disc).factor())
    # temp_list[0] = (P, n)
    ramified_primes_above = [item[0] for item in temp_list]
    
    # check for each ramified places
    result = True
    for P in ramified_primes_above:
        try:
            P_abs = P.absolute_ideal(w)
        except:
            print(F)
            print(F.base_field())
            
        abs_decomp = G_abs.decomposition_group(P_abs)
        rel_decomp = []
        for g in abs_decomp:
            if g(k_gen) == k_gen:
                rel_decomp.append(g)
        rel_decomp = abs_decomp.subgroup(rel_decomp)
        if not rel_decomp.is_cyclic():
            result = False
            break
    return result

## Definition 4.9.
Define the *patching degree* $\Delta_r$ of $U_r$ to be the maximum nonnegative integer $d \leq \Delta^{\omega}_r$ satisfying the following:
1. If $U_{>r}$ is nonempty, then the field $K_0(d) M_{U_{> r}}(d)$ is locally cyclic.
2. If $U_{<r}$ is nonempty, then the field $K_0(d) M_{U_r}(d)$ is locally cyclic.

If $U_r = \mathcal{I}$, we set $\Delta_r = \epsilon_0$.
We have $\Delta^{\omega}_r \geq \Delta_r \geq r$.

In [43]:
def Delta(r):
    U_r = U(r)
    if U_r == I:
        return epsilon[0]
    
    Delta_omega_r = Delta_omega(r)
    U_greater_r = U_greater(r)
    U_smaller_r = U_smaller(r)
    
    flag_1 = bool(len(U_greater_r) > 0)
    flag_2 = bool(len(U_smaller_r) > 0)
    
    for d in range(r, Delta_omega_r+1):
        d_satisfy = True
        
        if flag_1:
            M_U_greater_r_d = M_composite(U_greater_r, d)
            temp_field = K_cyclic_subfield(0, d).composite_fields(M_U_greater_r_d)[0]
            if not is_locally_cyclic(temp_field):
                d_satisfy = False
            
        if flag_2:
            M_U_r_d = M_composite(U_r, d)
            temp_field = K_cyclic_subfield(0, d).composite_fields(M_U_r_d)[0]
            if not is_locally_cyclic(temp_field):
                d_satisfy = False
                
        if not d_satisfy:
            return d-1
    return Delta_omega_r

A *bicyclic extension* $F/k$ is a Galois extension with $\mathrm{Gal}(F/k)$ isomorphic to $\mathbb{Z}/n_1 \mathbb{Z} \times \mathbb{Z}/n_2 \mathbb{Z}$ where $n_1, n_2 > 1$ and $n_2 \mid n_1$. (p2)

I think that this can be checked by looking at the numbers of elementary divisors of $\mathrm{Gal}(F/k)$: if there are more than $2$ elementary divisor then $F/k$ is not a bicyclic extension.

In [44]:
k.polynomial()

x^2 + 1

In [45]:
def is_bicyclic(F):
    L.<w> = F.absolute_field()
    G = L.galois_group()
    k_gal = []
    k_gen = k.polynomial().roots(ring = L, multiplicities = False)[0]
    for g in G:
        try:
            if g(k_gen) == k_gen:
                k_gal.append(g)
        except:
            print(f"I can't handle when F is the field {F}.")
    Gal_F_k = G.subgroup(k_gal)
    
    
    if Gal_F_k.is_abelian():
        Gal_fin_present = Gal_F_k.as_finitely_presented_group(reduced = True)
        gen_orders = [a for a in Gal_fin_present.abelian_invariants()]
        return len(AbelianGroup(gen_orders).elementary_divisors()) <= 2
    else:
        return False

In [46]:
is_bicyclic(K[0])

True

convert absolute field to relative number field

In [47]:
def abs_to_rel(F):
    R.<x> = PolynomialRing(k)
    p = R(F.polynomial())
    l = list(p.factor)
    F_rel.<v> = k.extension(l[0][0])
    return F_rel

## Definition 5.3
For a nonempty $U_r$, let $l_r = L(U_r)$. Let $f \leq \Delta^{\omega}_r$ be a nonnegative integer satisfying the following:
1. The field $M_{U_r}(f + l_r - r)$ is a subfield of a bicyclic extension.
2. $K_0(f) \subset M_{U_r}(f + l_r - r)$.
Then we set $f^{\omega}_{U_r}$ to be the largest $f \leq \Delta^{\omega}_r$ satisfying above conditions. We call $f^{\omega}_{U_r}$ the *algebraic degree of freedom* of $U_r$.

## Remark 5.4
We have $\Delta^{\omega}_r \geq f^{\omega}_{U_r} \geq r$.

For $h \geq L(U_r)$ and a class $c_0$ of $U_r/\sim_h$, we define by recursion *the algebraic degree of freedom* of $c \in c_0/\sim_{h+1}$ as follows.
## Definition 5.5
Keep the notation defined as above. Let $f \leq f^{\omega}_{c_0}$ be a non-negative integer satisfying the following:
1. The field $M_c(f + L(c) - r)$ is a subfield of a bicyclic extension.
2. $K_0(f) \subset M_c(f + L(c) - r)$.
Then we set $f^{\omega}_c$ to be the largest $f \leq f^{\omega}_{c_0}$ satisfying above conditions. We call $f^{\omega}_c$ the *algebraic degree of freedom* of $c$.

In [48]:
f_omega_c = {}

In [49]:
def alg_deg_of_freedom(c, r, c0 = None):
    # case 1: c == U_r for some r
    if c0 == None:
        U_r = c
        if len(U_r) > 0:
            l_r = L(U_r)
            Delta_omega_r = Delta_omega(r)
            
            f_omega_c[tuple(set(U_r))] = Delta_omega_r
            for f in range(r, Delta_omega_r + 1):
                # check temp_field can be defined:
                if min([epsilon[i] for i in U_r]) < f + l_r - r:
                    f_omega_c[tuple(set(U_r))] = f-1
                    break
                
                temp_field = M_composite(U_r, f+l_r-r)
                if is_bicyclic(temp_field) and is_subfield(K_cyclic_subfield(0, f), temp_field):
                    continue
                else:
                    f_omega_c[tuple(set(U_r))] = f-1
                    break
            
            # compute the next nevel
            if len(U_r) > 1:
                h = l_r
                for cl in equiv_classes(c, h+1):
                    alg_deg_of_freedom(cl, r, c)
            
    # case 2: c0 is a class of U_r
    if c0 != None:
        if len(c) > 0:
            L_c = L(c)
            
            f_omega_c[tuple(set(c))] = f_omega_c[tuple(set(c0))]
            for f in range(0, f_omega_c[tuple(set(c0))] + 1):
                # check temp_field can be defined:
                if min([epsilon[i] for i in c]) < f + L_c - r:
                    f_omega_c[tuple(set(c))] = f-1
                    break
                
                temp_field = M_composite(c, f + L_c - r)
                if is_bicyclic(temp_field) and is_subfield(K_cyclic_subfield(0, f), temp_field):
                    continue
                else:
                    f_omega_c[tuple(set(c))] = f-1
                    break
                    
            # compute the next level
            if len(c) > 1:
                h = L_c
                for cl in equiv_classes(c, h+1):
                    alg_deg_of_freedom(cl, r, c)

for r in range(0, epsilon[0]+1):
    alg_deg_of_freedom(U(r), r)

TypeError: unable to convert log(4)/log(2) + 1 to an integer

In [ ]:
f_omega_c

## Definition 5.6
For a nonempty $U_r$, let $h \geq L(U_r)$ and $c \in U_r/\sim_h$. We define the *degree of freedom* $f_c$ to be the maximum integer $f \leq f^{\omega}_c$ such that $M_c(f + L(c) - r)$ is locally cyclic.

In [ ]:
dic = {1: 2, 'key2': 'sage'}

In [ ]:
dic[1]

In [ ]:
c = [1, 2]

In [ ]:
type(c)

In [ ]:
c[0] = 3
c

In [ ]:
tup = (1, 2)
type(tup)
tup[0]

In [ ]:
tup[0] = 3

In [ ]:
# f_c = {(1, 2): 3, (0,): 4}

In [ ]:
f_c = {}

In [ ]:
def deg_of_freedom(c, r):
    c = tuple(set(c))
    if c in f_c:
        return f_c[c]
    else:
        for f in range(f_omega_c[c] + 1):
            temp_field = M_composite(c, f + L(c) - r)
            if is_locally_cyclic(temp_field):
                continue
            else:
                f_c[c] = f - 1
                return f_c[c]

## Theorem 6.5
We have
\begin{align*}
Ш^2_{\omega}(k, \hat{T}_{L/K}) \simeq \bigoplus\limits_{r \in \mathcal{R} \setminus \{0\}} \mathbb{Z}/p^{\Delta_r^{\omega} - r} \mathbb{Z} \bigoplus\limits_{r \in \mathcal{R}}
\bigoplus\limits_{l \geq L(U_r)} \bigoplus\limits_{c \in U_r/\sim_l} (\mathbb{Z}/p^{f^{\omega}_c - r} \mathbb{Z})^{n_{l+1}(c) -1}
\end{align*}
and
\begin{align*}
Ш^1(k, T_{L/K}) \simeq \bigoplus\limits_{r \in \mathcal{R} \setminus \{0\}} \mathbb{Z}/p^{\Delta_r - r} \mathbb{Z} \bigoplus\limits_{r \in \mathcal{R}} \bigoplus\limits_{l \geq L(U_r)} \bigoplus\limits_{c \in U_r/\sim_l} (\mathbb{Z}/p^{f_c - r} \mathbb{Z})^{n_{l+1}(c) -1}
\end{align*}

In [ ]:
[1]*5

In [ ]:
def sha2omega():
    first_part = []
    for r in R:
        if r == 0:
            continue
        if Delta_omega(r)-r >= 0:
            first_part.append(p^(Delta_omega(r)-r))

    second_part = []
    for r in R:
        U_r = U(r)
        l = L(U_r)
        keep_running = True
        while(keep_running):
            for c in equiv_classes(U_r, l):
                if f_omega_c[tuple(set(c))]-r >= 0:
                    second_part += [p^(f_omega_c[tuple(set(c))]-r)]*(n(l+1, c)-1)
                    
            if n(l, U_r) == len(U_r): # there is no need to use finer partition
                keep_running = False
            l += 1
    return AbelianGroup(first_part + second_part)

In [ ]:
sha2omega()

In [ ]:
first_part = []
for r in R:
    if r == 0:
        continue
    if Delta_(r)-r >= 0:
        first_part.append(p^(Delta(r)-r))

In [ ]:
first_part

In [ ]:
second_part = []

for r in R:
    U_r = U(r)
    l = L(U_r)
    keep_running = True
    while(keep_running):
        for c in equiv_classes(U_r, l):
            f_c = deg_of_freedom(c, r)
            if f_c-r >= 0:
                second_part += [p^(f_c-r)]*(n(l+1, c)-1)
        if n(l, U_r) == len(U_r): # there is no need to use finer partition
            keep_running = False
        l += 1